In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import model.cell as cell
import model.spec as spec
import numpy as np


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [2]:
np.shape(train_images)

(50000, 32, 32, 3)

In [8]:

cf = spec.Spec.get_configuration_space().sample_configuration()
s = spec.Spec(cf)
while s.valid_spec == False:
    cf = spec.Spec.get_configuration_space().sample_configuration()
    s = spec.Spec(cf)

c = cell.Cell(s, 128)

print(s.matrix)
print(s.ops)
c.compute_vertex_channels(128, 128, s.matrix)

[[0 1 1 1 0]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
['input', 'conv1x1-bn-relu', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'output']


[128, 64, 64, 64, 128]

In [11]:
# out = c.call(inputs=tf.keras.Input(shape=(32,32,3)))
c.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:

history = c.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels))

Train on 50000 samples, validate on 10000 samples
   32/50000 [..............................] - ETA: 22:10

ValueError: tf.function-decorated function tried to create variables on non-first call.